<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221109_ML_K_%EC%B5%9C%EA%B7%BC%EC%A0%91%EC%9D%B4%EC%9B%83_KNN%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 와인 품질 분류

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title 데이터 불러오기
# https://archive.ics.uci.edu/ml/datasets/wine
# https://www.kaggle.com/datasets/akhil0007/wine-data
file_url = 'https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/wine.csv'
df = pd.read_csv(file_url)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# 출력되는 데이터의 소수점 4자리까지만 보기
pd.options.display.float_format = '{:,.4f}'.format 
df.describe()   # KNN -> 거리기반, 숫자 단위에 민감 -> 스케일링 (전처리) 

In [ ]:
df.Class.value_counts()

In [ ]:
value_counts = df.Class.value_counts()
sns.barplot(x = value_counts.index, y = value_counts) 

In [ ]:
# https://plotly.com/python/
# barplot 숫자표시  
value_counts = df.Class.value_counts() 
bar = sns.barplot(x = value_counts.index, y = value_counts) 
for p in bar.patches:
    height = p.get_height() 
    bar.text(p.get_x() + p.get_width() / 2., height + 3, height, ha = 'center', size = 9) 
bar.set_ylim(-5, 100) 
plt.show() 

In [ ]:
# 데이터 특성 보기(info, describe) 
# 결측치 채워주기 
# 범주형 -> 더미 변수
# 훈련셋 / 시험셋 
# 스케일링 <= 훈련셋을 통해서 만들어진 스케일러로 시험셋 스케일링 
# 스케일링 이유: 아웃라이어 영향을 줄이거나 범위에 다른 거리의 왜곡 줄이기 
# 훈련하는 대상이 되는 데이터 <-> 시험을 대상으로 하는 데이터는 구분, 독립 
# 둘을 합쳐서 스케일링을 하면 최소-최대? 평균? 1, 2, 3분위(25, 50, 75%)  값이 공유
# 훈련셋을 대상으로만 스케일링함 
#    => 스케일러 학습 (훈련셋 학습 스케일러) 
#    => 시험셋도 이미 학습된(훈련셋 학습된 스케일러) 스케일러로 스케일링을 진행 

# 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
#@title 표준화 스케일링
# st_scaler = StandardScaler()
# st_scaler.fit(df)
# 스케일러가 fit()을 통해 학습한 정보를 통해 데이터를 변환(스케일링)하게 하는 함수
# st_scaled = st_scaler.transform(df)
# df_st_scaled = pd.DataFrame(st_scaled, columns = df.columns)
# df_st_scaled.describe() 

In [ ]:
# pd.options.display.float_format = '{:,.2f}'.format 
# df_st_scaled.describe() # mean(평균) = 0, std(표준편차) = 1

In [ ]:
#@title 로버스트스케일링
# rb_scaler = RobustScaler()
# fit + transform (한번에)

# rb_scalded = rb_scaler.fit_transform(df)
# df_rb_scaled = pd.DataFrame(rb_scalded, columns = df.columns)
# df_rb_scaled.describe()

In [ ]:
# #@title 최소최대 스케일링
# mm_scaler = MinMaxScaler()
# mm_scaled = mm_scaler.fit_transform(df)
# df_mm_scaled = pd.DataFrame(mm_scaled, columns = df.columns)
# df_mm_scaled.describe()

#### 스케일링 방식 선택 기준
* 아웃라이어의 유무 : 아웃라이어 영향이 클 시 로버스트 스케일링이 적합 (4분위값 이용)
* 데이터의 기존 분포를 최대한 유지해야할 시 : 최소-최대 스케일링
* 기존 데이터가 정규분포를 따르고 있고 아웃라이어가 없는 상황 : 표준화 스케일링

#### 스케일링 적용 시 주의점
* 스케일링 대상에서 종속변수를 제외
* 스케일링 전 훈련셋과 시험셋을 구분

In [ ]:
# 훈련셋과 시험셋 분리 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class', axis = 1),    # axis = 0 행, axis = 1 축 
                                                    df.Class, 
                                                    test_size = 0.2, random_state = 100)

In [ ]:
# 데이터 특성 보존을 위해 최대-최소 스케일러 사용 
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
mm_scaler = MinMaxScaler() 
mm_scaler.fit(X_train)

In [ ]:
X_train_scaled = mm_scaler.transform(X_train) 
X_train_scaled

In [ ]:
mm_scaler = MinMaxScaler()  # MinMaxScaler() 은 객체 , 객체는 안에 프로퍼티를 가지고 있음 
 # fit을 한다는건 특정한 데이터를 기준으로 학습을 시킴(프로퍼티라는 내부변수에 저장) 
 # fit 하고 transform (한번에 fit_transform) 이게 가능한건 mm_scaler.fit 이 내부에 저장되어 있기 때문에 X_test는 transform 만 해줘도 됨
X_train_scaled = mm_scaler.fit_transform(X_train)       
X_test_scaled = mm_scaler.transform(X_test) 
X_test_scaled

In [ ]:
#@title 모델링

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier() 
knn.fit(X_train_scaled, y_train) 
pred = knn.predict(X_test_scaled) 

In [ ]:
pred

In [ ]:
# 정확도보기 
from sklearn.metrics import accuracy_score 
accuracy_score(y_test, pred) 

In [ ]:
# 스케일링 안한 데이터 비교
knn2 = KNeighborsClassifier() 
knn2.fit(X_train, y_train) 
pred2 = knn2.predict(X_test) 
accuracy_score(y_test, pred2) 

In [ ]:
# 코드 진행 관련 함수화 하기 
# n은 정수형 int 를 패러미터로 받고, 최종 return 으로 실수형 float 를 반환해준다.
def tuning(n: int) -> float:
    knn = KNeighborsClassifier(n_neighbors = n) 
    knn.fit(X_train_scaled, y_train)
    pred = knn.predict(X_test_scaled)
    score = accuracy_score(y_test, pred)  
    return score  

In [ ]:
tuning(1)

In [ ]:
# for 문을 통해서 1 ~ 20까지 tuning 값 구하기 
for t in range(1, 21):
    print(t, tuning(t)) 

# for t in range(20):
    # print(t, tuning(t + 1))

# [t, tuning(t + 1) for t in range(20)] 

In [ ]:
k_list = [(t, tuning(t)) for t in range(1, 21)]

In [ ]:
sorted(k_list, key = lambda x: x[1], reverse = True) 

In [ ]:
import matplotlib.pyplot as plt

x = [4, 5, 10, 4, 3, 11, 14 , 8, 10, 12]
y = [21, 19, 24, 17, 16, 25, 24, 22, 21, 21]
classes = [0, 0, 1, 0, 0, 1, 1, 0, 1, 1]

plt.scatter(x, y, c=classes)